In [21]:
import pandas as pd
import numpy as np
import json
import pickle
import re

In [22]:
issuers = pd.read_excel('data/sentiment_dataset/issuers.xlsx')
issuers.head(2)

,Unnamed: 0,issuerid,EMITENT_FULL_NAME,datetrackstart,datetrackend,BGTicker,OtherTicker
0,0,1,"""Акционерный коммерческий банк ""Держава"" публи...",2021-06-02 12:47:55.100,NaN,NaN,NaN
1,1,2,"""МОСКОВСКИЙ КРЕДИТНЫЙ БАНК"" (публичное акционе...",2021-06-02 12:47:55.100,NaN,CBOM RX,NaN


In [23]:
syn = pd.read_excel('data/sentiment_dataset/names and synonyms.xlsx')
syn.head(2)

,issuerid,EMITENT_FULL_NAME,VeryOddCompany,BGTicker,BGTicker.1,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,1,"Акционерный коммерческий банк ""Держава"" публич...",NaN,NaN,DERZP,Держава,DERZHAVA,DERZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,"""МОСКОВСКИЙ КРЕДИТНЫЙ БАНК"" (публичное акционе...",NaN,CBOM RX,NaN,Московский кредитный банк,мкб,Credit Bank of Moscow,Credit Bank,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
names_texts = pd.read_csv('data\sentiment_dataset\mentions_text.csv')
names_texts.head(2)

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\karlo\AppData\Local\Temp\ipykernel_3944\671844907.py:1: SyntaxWarning: invalid escape sequence '\s'
  names_texts = pd.read_csv('data\sentiment_dataset\mentions_text.csv')


,Unnamed: 0,ChannelID,messageid,issuerid,MessageID,DateAdded,DatePosted,MessageText,IsForward
0,0,1197210433,5408,90,5408,2021-02-06 01:42:42,2020-04-29 07:29:01,?? Фокус недели #ФН Сегодня ????? ММК опублик...,False
1,1,1203560567,64803,57,64803,2021-02-06 01:47:00,2020-01-21 12:51:42,??#LSRG ЛСР - операционные результаты (2019г)...,False


In [25]:
sent_texts = pd.read_csv('data\sentiment_dataset\sentiment_text.csv')
sent_texts.head(2)

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\karlo\AppData\Local\Temp\ipykernel_3944\3656835817.py:1: SyntaxWarning: invalid escape sequence '\s'
  sent_texts = pd.read_csv('data\sentiment_dataset\sentiment_text.csv')


,Unnamed: 0,MessageID,ChannelID,issuerid,SentimentScore,DateAdded,DatePosted,MessageText,IsForward
0,0,241407,1203560567,153,2,2023-05-12 19:03:20,2023-05-12 19:02:42,⚠️🇷🇺#SELG #дивиденд сд Селигдар: дивиденды 20...,False
1,1,33684,1136626166,230,4,2023-02-03 20:56:29,2023-02-03 16:46:34,Ozon продолжает развивать специализированные ф...,False


In [26]:
uniq_sent = sent_texts['MessageText'].unique()
len(uniq_sent)

7188

In [27]:
d = {}
for i in range(len(uniq_sent)):
    d[uniq_sent[i]] = i

In [28]:
sent_texts['text_id'] = sent_texts['MessageText'].map(d)

In [29]:
i = 3
sent_texts.loc[sent_texts['text_id']==i]

,Unnamed: 0,MessageID,ChannelID,issuerid,SentimentScore,DateAdded,DatePosted,MessageText,IsForward,text_id
4,4,9826,1063908560,89,2,2023-04-24 17:51:38,2023-04-24 13:54:00,​​Windfall Tax — налог на сверхприбыль. Какие ...,False,3
5,5,9826,1063908560,127,2,2023-04-24 17:51:38,2023-04-24 13:54:00,​​Windfall Tax — налог на сверхприбыль. Какие ...,False,3
6,6,9826,1063908560,150,2,2023-04-24 17:51:38,2023-04-24 13:54:00,​​Windfall Tax — налог на сверхприбыль. Какие ...,False,3
7,7,9826,1063908560,7,2,2023-04-24 17:51:38,2023-04-24 13:54:00,​​Windfall Tax — налог на сверхприбыль. Какие ...,False,3


In [30]:
syn.loc[syn['issuerid']==220].T

,219
issuerid,220
EMITENT_FULL_NAME,Globaltrans Investment PLC (Глобалтранс Инвест...
VeryOddCompany,NaN
BGTicker,GLTR LI
BGTicker.1,NaN
Unnamed: 5,Globaltrans
Unnamed: 6,Глобалтранс
Unnamed: 7,NaN
Unnamed: 8,NaN
Unnamed: 9,NaN


In [31]:
uniq_sent[i]

'\u200b\u200bWindfall Tax — налог на сверхприбыль. Какие компании заплатят Минфину.  Многие понимают, что наш бюджет испытывает трудности и его необходимо наполнять всевозможными способами. Минфин использует различные финансовые рычаги: выход на долговой рынок — ОФЗ, продажа золота и юаней из ФНБ, но одним из главных является налоговый.   Как вы могли заметить, повышение НДПИ (налог на добычу полезных ископаемых) для различных отраслей в этом году уже состоялось. Причём некоторые заплатили разовый уже в прошлом и будут платить в этом: Газпром, Алроса. Это не считая того, что повысили процент на акцизы, штрафы, пошлины, тарифы и сборы.   Windfall Tax — совсем другая история.  Законопроект по налогу пока ещё не опубликован, но у заинтересованных лиц он уже есть на руках, его обсуждают различные издания и комментируют известные личности в финансовых кругах.   Налоговой базой для Windfall Tax будет превышение средней арифметической величины прибыли за 2021 год и за 2022 год над прибылью за

In [32]:
uniq_texts = names_texts['MessageText'].unique()
uniq_texts[0]

'?? Фокус недели #ФН  Сегодня ????? ММК опубликует финансовую отчётность за 1 кв. 2020 #MAGN #Отчетность ?????? Магнит представит операционные результаты за 1 кв. 2020 #MGNT #Результаты ?????? Новатэк опубликует финансовую отчётность за 1 кв. 2020 #NVTK #Отчетность ?????? ФРС США представит решение по процентной ставке #ФРС #Макро ?????? Tesla опубликует финансовую отчётность за 1 кв. 2020 #TSLA #Отчетность  ?????? Microsoft опубликует финансовую отчётность за 1 кв. 2020 #MSFT #Отчетность  ?????? Facebook опубликует финансовую отчётность за 1 кв. 2020 #FB #Отчетность  30 апреля ?????? Детский мир опубликует финансовую отчётность по МСФО за 2019 #DSKY #Отчетность ?????? Энел Россия опубликует финансовую отчётность по РСБУ за 1 кв. #ENRU #Отчетность ????? Evraz представит операционные результаты за 1 кв. 2020 #EVR #Результаты ????? НорНикель опубликует производственные результаты за 1 кв. 2020 #GMKN #Результаты ?????? РусГидро представит финансовую отчётность по РСБУ за 1 кв. 2020 #HYDR 

In [33]:
len(uniq_texts)

16611

In [34]:
uniq_texts[0]

'?? Фокус недели #ФН  Сегодня ????? ММК опубликует финансовую отчётность за 1 кв. 2020 #MAGN #Отчетность ?????? Магнит представит операционные результаты за 1 кв. 2020 #MGNT #Результаты ?????? Новатэк опубликует финансовую отчётность за 1 кв. 2020 #NVTK #Отчетность ?????? ФРС США представит решение по процентной ставке #ФРС #Макро ?????? Tesla опубликует финансовую отчётность за 1 кв. 2020 #TSLA #Отчетность  ?????? Microsoft опубликует финансовую отчётность за 1 кв. 2020 #MSFT #Отчетность  ?????? Facebook опубликует финансовую отчётность за 1 кв. 2020 #FB #Отчетность  30 апреля ?????? Детский мир опубликует финансовую отчётность по МСФО за 2019 #DSKY #Отчетность ?????? Энел Россия опубликует финансовую отчётность по РСБУ за 1 кв. #ENRU #Отчетность ????? Evraz представит операционные результаты за 1 кв. 2020 #EVR #Результаты ????? НорНикель опубликует производственные результаты за 1 кв. 2020 #GMKN #Результаты ?????? РусГидро представит финансовую отчётность по РСБУ за 1 кв. 2020 #HYDR 

In [35]:
text = uniq_texts[0]
text = re.sub(r'\?{2,}', '', text)
text = re.sub(r'\s{2,}', ' ', text)
text = re.sub(r'\@\S{2,}', '', text)
text = re.sub(r'http\S{2,}', '', text)
text = re.sub(r'\"{2,}', '"', text)
text

' Фокус недели #ФН Сегодня ММК опубликует финансовую отчётность за 1 кв. 2020 #MAGN #Отчетность Магнит представит операционные результаты за 1 кв. 2020 #MGNT #Результаты Новатэк опубликует финансовую отчётность за 1 кв. 2020 #NVTK #Отчетность ФРС США представит решение по процентной ставке #ФРС #Макро Tesla опубликует финансовую отчётность за 1 кв. 2020 #TSLA #Отчетность Microsoft опубликует финансовую отчётность за 1 кв. 2020 #MSFT #Отчетность Facebook опубликует финансовую отчётность за 1 кв. 2020 #FB #Отчетность 30 апреля Детский мир опубликует финансовую отчётность по МСФО за 2019 #DSKY #Отчетность Энел Россия опубликует финансовую отчётность по РСБУ за 1 кв. #ENRU #Отчетность Evraz представит операционные результаты за 1 кв. 2020 #EVR #Результаты НорНикель опубликует производственные результаты за 1 кв. 2020 #GMKN #Результаты РусГидро представит финансовую отчётность по РСБУ за 1 кв. 2020 #HYDR #Отчетность СД НЛМК рассмотрит дивиденды за 1 кв. 2020 #NLMK #Дивиденды Apple опубликуе

In [36]:
def clear_text(df):
    remove_question = lambda text: re.sub(r'\?{2,}', '', text)
    df['MessageTextClean'] = df['MessageText'].apply(remove_question)
    remove_space = lambda text: re.sub(r'\s{2,}', ' ', text)
    df['MessageTextClean'] = df['MessageTextClean'].apply(remove_space)
    remove_at = lambda text: re.sub(r'\@\S{2,}', '', text)
    df['MessageTextClean'] = df['MessageTextClean'].apply(remove_at)
    remove_site = lambda text: re.sub(r'http\S{2,}', '', text)
    df['MessageTextClean'] = df['MessageTextClean'].apply(remove_site)
    remove_quotation = lambda text: re.sub(r'\"{2,}', '"', text)
    df['MessageTextClean'] = df['MessageTextClean'].apply(remove_quotation)
    remove_u200b = lambda text: re.sub(r'\u200b', '', text)
    df['MessageTextClean'] = df['MessageTextClean'].apply(remove_u200b)
    return df

In [38]:
names_texts

,Unnamed: 0,ChannelID,messageid,issuerid,MessageID,DateAdded,DatePosted,MessageText,IsForward
0,0,1197210433,5408,90,5408,2021-02-06 01:42:42,2020-04-29 07:29:01,?? Фокус недели #ФН Сегодня ????? ММК опублик...,False
1,1,1203560567,64803,57,64803,2021-02-06 01:47:00,2020-01-21 12:51:42,??#LSRG ЛСР - операционные результаты (2019г)...,False
2,2,1197210433,23389,152,23389,2021-07-21 13:46:31,2021-07-21 11:15:46,#CHMF Северсталь (CHMF) впервые поставила в Бр...,False
3,3,1066174394,677,112,677,2021-09-21 04:23:59,2016-12-16 10:00:04,"""Версия: Многоходовочка по Роснефти Роснефтег...",False
4,4,1239405989,4486,115,4486,2023-01-20 15:02:22,2023-01-20 13:03:38,🟢 Новости к этому часу ⚪️ФРС США необходимо ...,False
...,...,...,...,...,...,...,...,...,...
19350,19350,1326656811,1758,116,1758,2021-02-06 19:05:52,2019-07-10 20:01:05,"""#NLMK """"Новолипецкий металлургический комбина...",False
19351,19351,1075101206,6196,218,6196,2021-02-06 01:02:00,2020-10-15 10:48:10,?? За 2020 год на МосБирже может появиться 10 ...,False
19352,19352,1203560567,141069,227,141069,2021-06-15 14:30:24,2021-06-04 08:01:55,"""????#FIVE #ритейлеры #россия СДЕЛКИ """"МАГНИТ...",False
19353,19353,1160971567,283,115,283,2021-09-21 04:33:17,2020-02-17 23:11:07,UPD: ТА Интер РАО UPD: ТА Новатэк,False


In [39]:
names_texts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19355 entries, 0 to 19354
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   19355 non-null  int64 
 1   ChannelID    19355 non-null  int64 
 2   messageid    19355 non-null  int64 
 3   issuerid     19355 non-null  int64 
 4   MessageID    19355 non-null  int64 
 5   DateAdded    19355 non-null  object
 6   DatePosted   19355 non-null  object
 7   MessageText  19354 non-null  object
 8   IsForward    19355 non-null  bool  
dtypes: bool(1), int64(5), object(3)
memory usage: 1.2+ MB


In [40]:
%%time
names_texts = clear_text(names_texts)
sent_texts = clear_text(sent_texts)

TypeError: expected string or bytes-like object, got 'float'

In [273]:
names_texts.head(2)

,ChannelID,messageid,issuerid,MessageID,DateAdded,DatePosted,MessageText,IsForward,MessageTextClean
0,1197210433,5408,90,5408,2021-02-06 01:42:42,2020-04-29 07:29:01,?? Фокус недели #ФН Сегодня ????? ММК опублик...,False,Фокус недели #ФН Сегодня ММК опубликует финан...
1,1203560567,64803,57,64803,2021-02-06 01:47:00,2020-01-21 12:51:42,??#LSRG ЛСР - операционные результаты (2019г)...,False,#LSRG ЛСР - операционные результаты (2019г) *п...


In [366]:
issuers['issuerid'].nunique()

255

In [367]:
syn['issuerid'].nunique()

255

In [30]:
names = issuers.merge(syn.drop(['EMITENT_FULL_NAME', 'BGTicker'], axis=1), on='issuerid', how='inner')
names.head(2)

,Unnamed: 0,issuerid,EMITENT_FULL_NAME,datetrackstart,datetrackend,BGTicker,OtherTicker,VeryOddCompany,BGTicker.1,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,0,1,"""Акционерный коммерческий банк ""Держава"" публи...",2021-06-02 12:47:55.100,NaN,NaN,NaN,NaN,DERZP,Держава,DERZHAVA,DERZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,"""МОСКОВСКИЙ КРЕДИТНЫЙ БАНК"" (публичное акционе...",2021-06-02 12:47:55.100,NaN,CBOM RX,NaN,NaN,NaN,Московский кредитный банк,мкб,Credit Bank of Moscow,Credit Bank,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
l_syns = ['EMITENT_FULL_NAME', 'BGTicker', 'OtherTicker', 'BGTicker.1']
for i in range(5, 15):
    l_syns.append('Unnamed: '+str(i))
l_syns

['EMITENT_FULL_NAME',
 'BGTicker',
 'OtherTicker',
 'BGTicker.1',
 'Unnamed: 5',
 'Unnamed: 6',
 'Unnamed: 7',
 'Unnamed: 8',
 'Unnamed: 9',
 'Unnamed: 10',
 'Unnamed: 11',
 'Unnamed: 12',
 'Unnamed: 13',
 'Unnamed: 14']

In [110]:
%%time
names['l_syns'] = names[l_syns].apply(lambda x: [re.sub(r'\s{2,}', '', a) for a in x if str(a) != 'nan'], axis=1)
names['l_syns'] = names['l_syns'].apply(lambda x: list(set(x)))
names['l_syns']

CPU times: total: 0 ns
Wall time: 4.45 ms


0      [Держава, DERZHAVA, "Акционерный коммерческий ...
1      [Credit Bank of Moscow, мкб, "МОСКОВСКИЙ КРЕДИ...
2      ["Российский акционерный коммерческий дорожный...
3      [алроса, alrosa, ALRS RX, Акционерная компания...
4      [Акционерный Коммерческий банк "АВАНГАРД" - пу...
                             ...                        
250                         [Хендерсон, HNFG, Henderson]
251                       [Совкомбанк, Sovcombank, SVCB]
252    [АЗС "Трасса", АЗС Трасса, EUTR, АЗС «Трасса»,...
253    ["Каршеринг Руссия", ПАО, Делимобил, Каршеринг...
254                             [Диасофт, Diasoft, DIAS]
Name: l_syns, Length: 255, dtype: object

In [111]:
names['l_syns'].loc[0]

['Держава',
 'DERZHAVA',
 '"Акционерный коммерческий банк "Держава" публичное акционерное общество"',
 'DERZP',
 'DERZ']

In [112]:
company_names = names[['issuerid', 'l_syns']].copy()
company_names.head()

,issuerid,l_syns
0,1,"[Держава, DERZHAVA, ""Акционерный коммерческий ..."
1,2,"[Credit Bank of Moscow, мкб, ""МОСКОВСКИЙ КРЕДИ..."
2,3,"[""Российский акционерный коммерческий дорожный..."
3,4,"[алроса, alrosa, ALRS RX, Акционерная компания..."
4,5,"[Акционерный Коммерческий банк ""АВАНГАРД"" - пу..."


In [113]:
company_names.to_pickle('company_names.pickle')

In [104]:
#company_names.to_csv('company_names.csv', index=False, sep=';')

# Разметка

In [374]:
def ff(x):
    for i in x:
        if ' rx' in i:
            x.append(i.split(' rx')[0])
            break
    return x

company_names = pd.read_pickle('company_names.pickle')
company_names['l_syns'] = company_names['l_syns'].apply(lambda x: [a.lower() for a in x])
company_names['l_syns'] = company_names['l_syns'].apply(lambda x: ff(x))
company_names['l_syns'].loc[1]

['credit bank of moscow',
 'мкб',
 '"московский кредитный банк" (публичное акционерное общество)',
 'cbom rx',
 'credit bank',
 'московский кредитный банк',
 'cbom']

In [375]:
names_texts.head()

,ChannelID,messageid,issuerid,MessageID,DateAdded,DatePosted,MessageText,IsForward,MessageTextClean
0,1197210433,5408,90,5408,2021-02-06 01:42:42,2020-04-29 07:29:01,?? Фокус недели #ФН Сегодня ????? ММК опублик...,False,Фокус недели #ФН Сегодня ММК опубликует финан...
1,1203560567,64803,57,64803,2021-02-06 01:47:00,2020-01-21 12:51:42,??#LSRG ЛСР - операционные результаты (2019г)...,False,#LSRG ЛСР - операционные результаты (2019г) *п...
2,1197210433,23389,152,23389,2021-07-21 13:46:31,2021-07-21 11:15:46,#CHMF Северсталь (CHMF) впервые поставила в Бр...,False,#CHMF Северсталь (CHMF) впервые поставила в Бр...
3,1066174394,677,112,677,2021-09-21 04:23:59,2016-12-16 10:00:04,"""Версия: Многоходовочка по Роснефти Роснефтег...",False,"""Версия: Многоходовочка по Роснефти Роснефтега..."
4,1239405989,4486,115,4486,2023-01-20 15:02:22,2023-01-20 13:03:38,🟢 Новости к этому часу ⚪️ФРС США необходимо ...,False,🟢 Новости к этому часу ⚪️ФРС США необходимо пр...


In [411]:
df = names_texts[['issuerid', 'MessageTextClean']].copy()

In [412]:
df.duplicated().sum()

115

In [413]:
df.drop_duplicates(inplace=True)

In [414]:
df.shape

(19240, 2)

In [415]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("russian")

def find_name(text, name, size):
    return [(m.start(), m.start()+size) for m in re.finditer(name, text)]

k = 0
def find_names(text, issuerid):
    global k
    #print('issuerid', issuerid)
    text = text.lower()
    d = {}
    d['text'] = text
    names = company_names.loc[company_names['issuerid']==issuerid, 'l_syns'].iloc[0]
    #print(names)
    l = []
    for name in names:
        l.extend(find_name(text, name, len(name)))
        name1 = stemmer.stem(name)
        if not name.__eq__(name1):
            l.extend(find_name(text, name1, len(name)))
    d['name'] = list(set(l))
    if d['name']:
        k += 1
    return d

n = 1500
text = names_texts.loc[n, 'MessageTextClean']
issuerid = names_texts.loc[n, 'issuerid']
d = find_names(text, issuerid)
d['text']

'алроса: есть ли перспективы у компании? недавно компания представила сильный отчет по итогам 2023 г. алроса продолжает реализовывать камни по неплохим для нее ценам, несмотря на множество введенных санкций в отношении нее. в этом посте обсудим финансовые результаты компании, цены на алмазы и влияние санкций на ее операционную деятельность. также дадим актуальный взгляд на алросу. финансовые результаты по итогам 2023 г. — выручка выросла на 9% г/г и составила 322.6 млрд руб. компания должна была прекратить продажи алмазов на 2 месяца (сентябрь-ноябрь), однако выручка за последнее полугодие была высокой. вероятно, продажи были остановлены только на 1-1.5 месяца. прибыль от продаж снизилась на 10% г/г, до 106.6 млрд руб., а чистая прибыль — на 15%, до 85.2 млрд руб. — fcf составил 2.2 млрд руб., тогда как мы ожидали 14.5 млрд руб. капитальные инвестиции были на 50% выше нашего прогноза и составили 63.6 млрд руб. — мы ожидаем, что по итогам 2023 г. компания заплатит 5.75 руб. дивидендов (

In [416]:
d['name']

[(1604, 1610),
 (1910, 1916),
 (101, 107),
 (0, 6),
 (2505, 2511),
 (2753, 2759),
 (375, 381)]

In [417]:
for i in range(len(d['name'])):
    print(d['text'][d['name'][i][0]:d['name'][i][1]])

алросы
алроса
алроса
алроса
алросы
алросы
алросу


In [418]:
from tqdm import tqdm

pp = 0
rez = []
for i in tqdm(df.index):
    z = df.loc[i]
    if z['issuerid'] in company_names['issuerid'].to_numpy():
        rez.append(find_names(z['MessageTextClean'], z['issuerid']))
    else:
        pp += 1

100%|██████████████████████████████████████████████████████████████████████████| 19240/19240 [00:15<00:00, 1238.37it/s]


In [419]:
k

18005

In [420]:
df.shape[0]

19240

In [421]:
pp

4

In [422]:
18025 / 19244

0.9366555809602993

In [423]:
rez[3]

{'text': '"версия: многоходовочка по роснефти роснефтегаз размещает депозит до 1,8 трлн руб в газпромбанке, газпромбанк через intesa sanpaolo финансирует кредитами glencore и катарский фонд qia, glencore и qia покупают у роснефтегаза пакет акций роснефти у роснефти ещё есть подконтрольный ему "карманный" банк вбрр, который также мог иметь взаимоотношения с intesa sanpaolo газетная версия ',
 'name': [(247, 255), (236, 244), (211, 219), (27, 35), (36, 44)]}

In [424]:
with open('data_clean.json', 'w') as f:
    json.dump({'data': rez}, f)

In [452]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("russian")

def find_name(text, name, size):
    return [(m.start(), m.start()+size) for m in re.finditer(name, text)]

k = 0
def find_company_names(text, issuerid):
    #print('issuerid', issuerid)
    text = text.lower()
    d = {}
    d['text'] = text
    names = company_names.loc[company_names['issuerid']==issuerid, 'l_syns'].iloc[0]
    #print(names)
    l = []
    for name in names:
        l.extend(find_name(text, name, len(name)))
        name1 = stemmer.stem(name)
        if not name.__eq__(name1):
            l.extend(find_name(text, name1, len(name)))
    return list(set(l))

def find_companys_names(text):
    global k
    subdf = df.loc[df['MessageTextClean']==text]
    l = []
    d = {}
    for issuerid in subdf['issuerid']:
        print(issuerid)
        ll = find_company_names(text, issuerid)
        if not ll: return d
        l.extend(ll)
    k += 1
    d['text'] = text
    d['name'] = l
    return d

find_companys_names(df.loc[11, 'MessageTextClean'])

227


{}

In [453]:
df.loc[11, 'MessageTextClean']

' Главные новости за 20 июля: Итоги торгов. Показали уверенный рост опережающими темпами Рынок США. В фокусе - отчетность, коронавирус, стимулы Ключевые события недели: заседание ЦБ РФ, отчеты Tesla, Coca-Cola, июльские PMI X5Retail создает сеть пунктов выдачи. Что это значит для АФК Системы Chevron покупает Noble Energy за $5 млрд Ключевой акционер Петропавловска конвертирует облигации в акции. Что это значит Какие риски у доллара ВТБ - лидер дня'

In [454]:
df.loc[df['MessageTextClean']==df.loc[11, 'MessageTextClean']]

,issuerid,MessageTextClean
10,227,Главные новости за 20 июля: Итоги торгов. Пок...
11,26,Главные новости за 20 июля: Итоги торгов. Пок...


In [ ]:
pp = 0
rez = []
for i in tqdm(df.index):
    z = df.loc[i]
    if z['issuerid'] in company_names['issuerid'].to_numpy():
        rez.append(find_names(z['MessageTextClean'], z['issuerid']))
    else:
        pp += 1